In [1]:
# !pip install llama-index
# !pip install GPT4All
# !pip install einops
# !pip install pypdf
# !pip install sentence-transformers



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import spacy
from typing import List

nlp = spacy.load('es_core_news_lg')

def remove_stop_words(text: str) -> str:
  doc = nlp(text)
  text_parts = [token.text for token in doc if not token.is_stop]
  return "".join(text_parts)

def split_sentences(text: str) -> List[str]:
  doc = nlp(text)
  sentences = [sent.text for sent in doc.sents]
  return sentences

def group_sentences_semantically(sentences: List[str], threshold: int) -> List[str]:
  docs = [nlp(sentence) for sentence in sentences]
  segments = []

  start_idx = 0
  end_idx = 1
  segment = [sentences[start_idx]]
  while end_idx < len(docs):
    if docs[start_idx].similarity(docs[end_idx]) >= threshold:
      segment.append(docs[end_idx])
    else:
      segments.append(" ".join(segment))
      start_idx = end_idx
      segment = [sentences[start_idx]]
    end_idx += 1

  if segment:
    segments.append(" ".join(segment))

  return segments

def split_text(text: str) -> List[str]:
  text_no_stop_words = remove_stop_words(text)
  sentences = split_sentences(text_no_stop_words)
  return group_sentences_semantically(sentences, 0.8)

In [23]:

# Initialize the cross-encoder model


# Define the two documents
doc1 = "The University of Havana is one of the most important academic institutions in Cuba"
doc2 = "There are many universities In the city of Havana, many of them are the best universities in the country" 
# doc2 = "Havana University has a very influent AI research group"
# doc2 = "Holguin is a nice city"


# Compute the semantic similarity
scores = model.predict([[doc1, doc2]])

# Print the similarity score
print(f"The semantic similarity between the two documents is: {scores}")

The semantic similarity between the two documents is: [0.6647494]


{'examples': [<sentence_transformers.readers.InputExample.InputExample at 0x7f9a71dc3a50>,
  <sentence_transformers.readers.InputExample.InputExample at 0x7f9a74d25010>]}

In [2]:
# model = 'llama-2-7b-chat.ggmlv3.q4_0.bin
model = 'ggml-model-gpt4all-falcon-q4_0.bin'
# model = 'gpt4all-lora-quantized.bin'

local_path = (
    f"./models/{model}"  # replace with your desired local file path
)

import requests

from pathlib import Path
from tqdm import tqdm

Path(local_path).parent.mkdir(parents=True, exist_ok=True)

if not os.path.exists(local_path):
  url = f'http://gpt4all.io/models/{model}'
  response = requests.get(url, stream=True)

  with open(local_path, 'wb') as f:
      for chunk in tqdm(response.iter_content(chunk_size=8192)):
          if chunk:
              f.write(chunk)

NameError: name 'os' is not defined

In [ ]:
from langchain.schema import language_model
from langchain.llms import gpt4all
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding, ServiceContext, OpenAIEmbedding
from llama_index.llms import LangChainLLM, HuggingFaceLLM
from langchain.llms import gpt4all

import torch
from transformers import BertTokenizerFast, EncoderDecoderModel
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# ckpt = 'mrm8488/bert2bert_shared-spanish-finetuned-summarization'
# ckpt = 'google/pegasus-cnn_dailymail'
# tokenizer = BertTokenizerFast.from_pretrained(ckpt)
# model = EncoderDecoderModel.from_pretrained(ckpt).to(device)


# from llama_index.prompts.prompts import SimpleInputPrompt

# This will wrap the default prompts that are internal to llama-index
# taken from https://huggingface.co/Writer/camel-5b-hf
# query_wrapper_prompt = SimpleInputPrompt(
#     "Below is an instruction that describes a task. "
#     "Write a response that appropriately completes the request.\n\n"
#     "### Instruction:\n{query_str}\n\n### Response:"
# )

# ckpt="Writer/camel-5b-hf"
model = gpt4all.GPT4All(model=f"./models/{model}")

model = LangChainLLM(llm=model)
# model =  
# model = HuggingFaceLLM(model=model)

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)


service_context = ServiceContext.from_defaults(
    llm=model,
    embed_model = embed_model,
    chunk_size=150
)

: 

# Memorándum del Seminario sobre Violencia y Paz al Consejo Asesor de la Comisión Ejecutiva de Atención a Víctimas

https://rddm.mx/busqueda/#/docs?id=q524jq22z&has_model=ArchivalDocument&thumbnail=%2Fdownloads%2F1831cm38k%3Ffile%3Dthumbnail&related=1831cm38k

In [7]:
from transformers import pipeline
import PyPDF2
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import MarianMTModel, MarianTokenizer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

pdf_path = "./documents/FCE_0001_00060.pdf"
text = ""


def translate_es_en(spanish_text):

    # Load the pre-trained model and tokenizer
    model_name = "Helsinki-NLP/opus-mt-es-en"  # Spanish to English translation model
    model = MarianMTModel.from_pretrained(model_name).to(device)
    tokenizer = MarianTokenizer.from_pretrained(model_name)

    # Tokenize the Spanish text
    inputs = tokenizer.encode(spanish_text, return_tensors="pt").to(device)

    # Generate translation
    translated_ids = model.generate(inputs, max_length=100, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
    translated_text = tokenizer.decode(translated_ids[0], skip_special_tokens=True)
    return translated_text

def translate_en_es(english_text):
    # Load the pre-trained model and tokenizer
    model_name = "Helsinki-NLP/opus-mt-en-es"  # English to Spanish translation model
    model = MarianMTModel.from_pretrained(model_name).to(device)
    tokenizer = MarianTokenizer.from_pretrained(model_name)

    # Tokenize the English text
    inputs = tokenizer.encode(english_text, return_tensors="pt").to(device)

    # Generate translation
    translated_ids = model.generate(inputs, max_length=100, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
    translated_text = tokenizer.decode(translated_ids[0], skip_special_tokens=True)

    return translated_text

def detect_topics_all(input):
    mname = "cristian-popa/bart-tl-all"
    tokenizer = AutoTokenizer.from_pretrained(mname)
    model = AutoModelForSeq2SeqLM.from_pretrained(mname).to(device)

    # input = "site web google search website online internet social content user"
    enc = tokenizer(input, return_tensors="pt", truncation=True, padding="max_length", max_length=1024).to(device)
    outputs = model.generate(
        input_ids=enc.input_ids,
        attention_mask=enc.attention_mask,
        max_length=25,
        min_length=1,
        do_sample=False,
        num_beams=25,
        length_penalty=1.0,
        repetition_penalty=1.5
    )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print("Topic:", decoded)
    return decoded


def detect_topics_ng(input):
    mname = "cristian-popa/bart-tl-ng"
    tokenizer = AutoTokenizer.from_pretrained(mname)
    model = AutoModelForSeq2SeqLM.from_pretrained(mname).to(device)

    # input = "site web google search website online internet social content user"
    enc = tokenizer(input, return_tensors="pt", truncation=True, padding="max_length", max_length=1024).to(device)
    outputs = model.generate(
        input_ids=enc.input_ids,
        attention_mask=enc.attention_mask,
        max_length=25,
        min_length=1,
        do_sample=False,
        num_beams=25,
        length_penalty=1.0,
        repetition_penalty=1.5
    )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print("Topic:", decoded)
    return decoded

In [8]:
import spacy

nlp_es = spacy.load('es_core_news_lg')

def filter_tokens(doc):
    
    doc = nlp_es(doc)
    # print(len(doc))
    filtered_tokens = [token.text for token in doc if not token.is_oov]
    # print(len(filtered_tokens))
    return " ".join(filtered_tokens)

In [3]:
import PyPDF2
def get_text():
    pdf_path = "./documents/FCE_0001_00060.pdf"
    
    text = ""
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        num_pages = len(pdf_reader.pages)
        
        topics = []
        
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text
    # return filter_tokens(text)            

In [10]:
from transformers import AutoTokenizer 

model_name = "Helsinki-NLP/opus-mt-en-es"  # English to Spanish translation model
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Load tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained("your-transformer-model-name")
# model = AutoModel.from_pretrained("your-transformer-model-name")

# Define maximum sequence length and batch size
max_seq_length = 150 # Adjust as needed
max_sum_length = 70
# Tokenize the text
text = get_text().split()

i = 0
batched_text = []

for start_idx in range(0, len(text), max_seq_length):
    sublist = text[start_idx:start_idx + max_seq_length]
    batched_text.append(sublist)
    
pending_text = ""
topics = set()
for i in batched_text:
    text = " ".join(i)
    en_text = translate_es_en(text)
    pending_text += en_text
    
    if len(pending_text.split()) < max_sum_length:
        continue
    
    text_to_summarize = pending_text
    summary = summarizer(text_to_summarize, max_length=max_sum_length, min_length=30, do_sample=False)
    es_text = translate_en_es(summary[0]["summary_text"])
    
    pending_text = ""


Topic: the concept
Las masacres de San Fernando, Tamaulipas y Allende, Coahuila. Lecciones para el Estado mexicano, la sociedad organizada y las víctimas.
Topic_ng

----------
Topic: human rights
Se han realizado esfuerzos para acceder a la información de la Procuraduría General de Justicia (PGR), la Comisión Nacional de Derechos Humanos (CNDH), las comisiones de derechos humanos Tamaulipas y Coahuila, y las fiscalías locales.
Topic_ng

----------
Topic: relevant
El análisis deja al Estado mexicano muy mal. En general, aunque su grado y forma de responsabilidad varía, las autoridades no se comportan correctamente, y también ha dado seguimiento a los pasos que la víctima debe seguir ante la comunidad internacional.
Topic_ng

----------
Topic: human rights
Las organizaciones de la sociedad civil han viajado de P a a g i n a 2 de una actitud cautelosa pero receptiva a otra colaboración. Muestra de esto son los contactos establecidos con la Fundación por la Justicia y el Estado Democrático

In [ ]:
from llama_index.file.base import SimpleDirectoryReader
from llama_index import GPTVectorStoreIndex

# other way of loading
# from llama_index import download_loader
# SimpleDirectoryReader = download_loader("SimpleDirectoryReader")

loader = SimpleDirectoryReader('./data', recursive=True, exclude_hidden=True)
documents = loader.load_data()
index = GPTVectorStoreIndex.from_documents(documents)
index.query('What are these files about?')

In [ ]:
from llama_hub.file.base import SimpleDirectoryReader
from llama_index import GPTVectorStoreIndex
from langchain.agents import initialize_agent, Tool
from llama_index.llms import LangChainLLM, HuggingFaceLLM
from langchain.llms import gpt4all
from langchain.chains.conversation.memory import ConversationBufferMemory


# other way of loading
# from llama_index import download_loader
# SimpleDirectoryReader = download_loader("SimpleDirectoryReader")

loader = SimpleDirectoryReader('./data', recursive=True, exclude_hidden=True)
documents = loader.load_data()
index = GPTVectorStoreIndex.from_documents(documents)

tools = [
    Tool(
        name="Local Directory Index",
        func=lambda q: index.query(q),
        description=f"Useful when you want answer questions about the files in your local directory.",
    ),
]
model = gpt4all.GPT4All(model=f"./models/{model}")

model = LangChainLLM(llm=model)
# model =  
# model = HuggingFaceLLM(model=model)

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)


service_context = ServiceContext.from_defaults(
    llm=model,
    embed_model = embed_model,
    chunk_size=150
)

memory = ConversationBufferMemory(memory_key="chat_history")
agent_chain = initialize_agent(
    tools, llm, agent="zero-shot-react-description", memory=memory
)

output = agent_chain.run(input="What are these files about?")

In [1]:

tgt_text


/home/joramas/miniconda/envs/thesis/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/joramas/miniconda/envs/thesis/lib/python3.11/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (64) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['Command K is a tool that lets you write code with an artificial intelligence (AI) program.',
 "California's largest electricity provider has turned off power to hundreds of thousands of customers."]

In [6]:
from transformers import AutoTokenizer, PegasusForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-large")
model = PegasusForCausalLM.from_pretrained("google/pegasus-large", add_cross_attention=False)
assert model.config.is_decoder, f"{model.__class__} has to be configured as a decoder."
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

logits = outputs.logits
expected_shape = [1, inputs.input_ids.shape[-1], model.config.vocab_size]
tokenizer.decode(outputs["input_ids"])

Some weights of PegasusForCausalLM were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: argument 'ids': 'dict' object cannot be converted to 'Sequence'

In [1]:
from transformers import AutoTokenizer, PegasusXForConditionalGeneration
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model = PegasusXForConditionalGeneration.from_pretrained("google/pegasus-x-large")
model.to(device)
tokenizer = AutoTokenizer.from_pretrained("google/pegasus-x-large")


/home/joramas/miniconda/envs/thesis/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:

inputs = tokenizer(ARTICLE_TO_SUMMARIZE,max_length=10000, truncation=True, return_tensors="pt").to(device)

# Generate Summary
print("GENERATING SUMMARY")
summary_ids = model.generate(inputs["input_ids"], max_length=1000)
tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]


SyntaxError: invalid syntax (2846213366.py, line 1)

In [4]:


try:
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        num_pages = len(pdf_reader.pages)
        
        topics = []
        
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            page_text = page.extract_text()
            
            
            for p in page_text.split("\n "):
                # if len(temp_text) > 512:
                p_es = p
                p = translate_es_en(p)
                print(len(p.split()))
                if len(p.split()) > 50:
                    topic = detect_topics(p)
                    text_to_summarize = f"Topic: {topic} {p}"
                    output = summarizer(p, max_length=45, min_length=10, do_sample=False)[0]["summary_text"] + "\n"
                    topics.append(topic)
                    print("Topic", topic)
                    output = translate_en_es(output)
                    text += output
                else:
                    print(p)
                    print(p_es)
            # if 0 < len(temp_text) > 130:
            #    text += summarizer(temp_text, max_length=130, min_length=30, do_sample=False) + "\n"
            
            # print("Page", page_num + 1, ":", page_text)
except FileNotFoundError:
    print("File not found.")
except Exception as e:
    print("An error occurred:", str(e))
    

An error occurred: name 'pdf_path' is not defined
